# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

In [9]:
import dask.array as da
import numpy as np
import h5py

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [5]:
arr_np = np.random.normal(0, 1, size=(1000, 300000))
arr_np

array([[-0.45962983, -1.26360917, -0.08618747, ...,  0.88187849,
         0.07789392, -0.52379956],
       [ 0.85261394, -2.37427948, -1.96694004, ..., -0.76867526,
         1.03121515, -1.50210403],
       [ 1.78691705, -0.28866219, -1.25863301, ...,  0.14151955,
         1.92708894, -1.55097616],
       ...,
       [-0.47147323,  1.42427213,  0.64303714, ..., -0.103971  ,
         0.1339361 , -1.35590844],
       [ 0.18527365,  0.14350049, -1.1430615 , ...,  1.18804379,
        -1.10847827,  0.57601394],
       [ 1.02992865, -0.17555795, -0.57408659, ...,  0.18487028,
        -0.70004159, -0.15826668]])

In [6]:
arr_da = da.random.normal(0, 1, size=(1000, 300000), chunks=(1000, 300000))
arr_da

C:\ProgramData\Anaconda3\lib\site-packages\dask\array\core.py:1401: RuntimeWarning: overflow encountered in long_scalars
  cbytes = format_bytes(np.prod(self.chunksize) * self.dtype.itemsize)


dask.array<normal, shape=(1000, 300000), dtype=float64, chunksize=(1000, 300000), chunktype=numpy.ndarray>

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [7]:
%%time
np.power(arr_np, 2).sum()

Wall time: 7.59 s


299999336.43839073

In [8]:
%%time
da.power(arr_dask, 2).sum().compute()

Wall time: 13.8 s


300004435.1564171

3. Визуализируйте граф вычислений для задачи 2.

## Лабораторная работа 11

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [53]:
hdf = h5py.File('minutes_n_ingredients_full.hdf5', 'r')
recipe = da.from_array(hdf['recipe'], chunks=(100000, 3))

In [54]:
recipe

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [55]:
recipe[:, 1:].mean(axis=0).compute()

array([1004.20805176,    5.4198008 ])

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [56]:
%%time
recipe[:, 1:].mean(axis=0).compute()

Wall time: 55.2 ms


array([1004.20805176,    5.4198008 ])

In [60]:
%%time
recipe1 = da.from_array(hdf['recipe'], chunks=(1000, 3))
recipe1[:, 1:].mean(axis=0).compute()

Wall time: 1.34 s


array([1004.20805176,    5.4198008 ])

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [79]:
da.median(recipe[:, 1], axis=0).compute()

32.0

In [80]:
recipe[recipe[:, 1] < da.median(recipe[:, 1], axis=0)].compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]], dtype=int64)

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [116]:
cnt = da.bincount(recipe[:, 2]).compute()
i = np.nonzero(cnt)[0]
da.vstack((i, cnt[i])).T.compute()

array([[     1, 222071],
       [     2, 224158],
       [     3, 229388],
       [     4, 234948],
       [     5, 240720],
       [     6, 244360],
       [     7, 247181],
       [     8, 246747],
       [     9, 246816],
       [    10,  22430],
       [    11,  19094],
       [    12,  15165],
       [    13,  11640],
       [    14,   8284],
       [    15,   6014],
       [    16,   4145],
       [    17,   2793],
       [    18,   1913],
       [    19,   1279],
       [    20,    852],
       [    21,    529],
       [    22,    346],
       [    23,    244],
       [    24,    178],
       [    25,    107],
       [    26,     68],
       [    27,     55],
       [    28,     33],
       [    29,     22],
       [    30,     20],
       [    31,     13],
       [    32,      5],
       [    33,      4],
       [    34,      3],
       [    35,      4],
       [    36,      1],
       [    37,      2],
       [    38,      1],
       [    39,      1],
       [    40,      2],


6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [73]:
max_time = recipe[recipe[:, 1] == recipe[:, 1].max(axis=0), 1].compute()
max_time

array([2147483647], dtype=int64)

In [88]:
type(recipe[:, 1])

dask.array.core.Array

In [131]:
quan = da.percentile(recipe[:, 1], q =75).compute()
s = recipe[recipe[:, 1] > quan, 1].compute() 


array([60, 57, 62, ..., 61, 60, 64], dtype=int64)

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [92]:
pref = da.array([15, 6])
pref

dask.array<array, shape=(2,), dtype=int32, chunksize=(2,), chunktype=numpy.ndarray>

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент